In [34]:
# Source: https://github.com/fmidev/opendata-resources/blob/master/examples/python/FMI_WFS2_getobs_multipointcoverage_example.ipynb

In [35]:
import requests
import datetime as dt
import xml.etree.ElementTree as ET
import numpy as np
import re
# import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import colorbar, colors

In [36]:
def get_param_names(url):
    """ Get parameters metadata"""
    req = requests.get(url)
    params = {}
    
    if req.status_code == 200:
        xmlstring = req.content
        tree = ET.ElementTree(ET.fromstring(xmlstring))                
        for p in tree.iter(tag='{http://inspire.ec.europa.eu/schemas/omop/2.9}ObservableProperty'):
            params[p.get('{http://www.opengis.net/gml/3.2}id')] = p.find('{http://inspire.ec.europa.eu/schemas/omop/2.9}label').text
    return params        
    
def get_params(tree):
    """ Get parameters from response xml tree """
    
    retParams = []
    for el in tree.iter(tag='{http://www.opengis.net/om/2.0}observedProperty'):
        url = el.get('{http://www.w3.org/1999/xlink}href')
        params = re.findall(r"(?<=param=).*,.*(?=&)", url)[0].split(',')

        param_names = get_param_names(url)
        for p in params:
            retParams.append('{} ({})'.format(param_names[p], p))
                
    return retParams

In [37]:
def get_positions(tree):
    """ 
    Function to get times and coordinates from multipointcoverage answer
    """
    positions = []
    for el in tree.iter(tag='{http://www.opengis.net/gmlcov/1.0}positions'):
        pos = el.text.split()
        i = 0
        while len(pos) > 0:
            lat = float(pos.pop(0))
            lon = float(pos.pop(0))
            timestamp = int(pos.pop(0))
            positions.append([lat,lon,timestamp])
    return np.array(positions)


In [51]:
url = 'http://opendata.fmi.fi/wfs'
bbox = '19,59,30,75'
data = False

starttime = dt.datetime.strptime('2010-01-01', "%Y-%m-%d")
endtime = dt.datetime.strptime('2010-01-03', "%Y-%m-%d")
daystep = 1

start = starttime
end = start + dt.timedelta(days=daystep)
if end > endtime: end = endtime
    
while end <= endtime and start < end:
    startStr = start.strftime('%Y-%m-%d')
    endStr = end.strftime('%Y-%m-%d')
    
    # Get data
    payload = {
        'request': 'getFeature',
        'storedquery_id': 'fmi::observations::weather::multipointcoverage',
        'parameters': 'windspeedms,winddirection,cloudamount',      # cloud amount parameter
        'bbox': bbox,
        'starttime': startStr,
        'endtime': endStr,    
    }
    r = requests.get(url, params=payload)
    
    # Construct XML tree
    tree = ET.ElementTree(ET.fromstring(r.content))

    # Get geospatial and temporal positions of data elements
    positions = get_positions(tree)
        
    # Extract data from XML tree
    d = []
    for el in tree.iter(tag='{http://www.opengis.net/gml/3.2}doubleOrNilReasonTupleList'):
        for pos in el.text.strip().split("\n"):
            d.append(list(map(float, pos.strip().split(' '))))
    
    # Assign data values to positions
    junk = np.append(positions, np.array(d), axis=1)
    if data is not False:
        data = np.append(data, junk, axis=0)
    else:
        data = junk
    
    print('Time interval {} - {} provided {} rows'.format(startStr, endStr, junk.shape[0]))
    
    start = end
    end = start + dt.timedelta(days=daystep)
    if end > endtime: end = endtime

print('Done fetching data. Final dimensions of the result: {}'.format(data.shape))
print(positions.shape)

AxisError: axis 1 is out of bounds for array of dimension 1

In [39]:
params = get_params(tree)

In [40]:
"""
fig = plt.figure(figsize=(18, 13))
first_timestamp = data[0,2]
axes = {}

for i in np.arange(0,4):    
    axes[i] = plt.subplot(2, 2, i+1, projection=ccrs.PlateCarree())
    axes[i].coastlines()

    timestamp = first_timestamp + 12*3600*i
    
    coords = data[(data[:,2] == timestamp)][:,0:2]
    coords = np.column_stack((coords[:,1], coords[:,0]))

    val = data[(data[:,2] == timestamp)][:,3]

    cb = axes[i].scatter(coords[:,0], coords[:,1], c=val, cmap='coolwarm')
    plt.colorbar(cb, cmap='coolwarm', orientation='vertical',ticklocation='auto')
    
    axes[i].set_title('{}'.format(dt.datetime.fromtimestamp(timestamp)))
"""
"""
dates = set()
for timestamp in data[:,2]:
    d = dt.datetime.fromtimestamp(timestamp)
    if d not in dates:
        dates.add(d)
        print(d)
"""

'\ndates = set()\nfor timestamp in data[:,2]:\n    d = dt.datetime.fromtimestamp(timestamp)\n    if d not in dates:\n        dates.add(d)\n        print(d)\n'